# Introduction to MLflow

Welcome! This notebook is a comprehensive guide to using MLflow, especially for Machine Learning projects. MLflow provides powerful tools to manage the machine learning lifecycle effectively.

## What is MLflow?

MLflow is an open-source platform designed for managing the entire machine learning lifecycle. It's beneficial for handling complex workflows in Machine Learning projects due to its flexible design and integration capabilities.

### Why is it useful for Machine Learning?

- **Experimentation Needs**: ML projects require extensive experimentation, and MLflow helps track and log all research efforts efficiently.
- **Model Complexity**: MLflow's features in model versioning and deployment simplify handling sophisticated models.
- **Collaborative Development**: It facilitates seamless teamwork among data scientists, engineers, and stakeholders.

## Key Features and Benefits for Machine Learning

### 1. Experiment Tracking
MLflow’s tracking API captures complex experiments, including model parameters, versions, and outcomes.

- **Benefits**:
  - Simplifies experimentation and comparative analysis between models.
  - Enhances reproducibility, a critical requirement in ML research.

### 2. Projects
The structured ML project format ensures consistency and reusability, necessary for iterative development.

### 3. Models
MLflow supports various deployment environments and formats, easing the deployment of ML solutions.

- **Benefits**:
  - Manages models through different lifecycle stages efficiently.
  - Supports large-scale deployments on cloud platforms.

### 4. Model Registry
MLflow’s model registry acts as a central hub, managing model versions.

- **Benefits**:
  - Organizes the model lifecycle seamlessly.
  - Facilitates integration with CI/CD pipelines, ideal for agile workflows.

## Architecture and Core Components

MLflow’s architecture supports comprehensive ML workflows through:

- **Tracking Server**: Handles comprehensive logs and metadata for ML experiments.
- **Backend Store**: Saves metadata across experiments, optimizing accessibility and scalability.
- **Artifact Store**: Manages larger artifacts like trained models and datasets used in ML projects.

# MLflow Feature Exploration: Experiment Tracking

Experiment tracking is crucial for managing and optimizing ML experiments.

## Experiment Tracking Overview

Logging and visualizing parameters, metrics, and other data generated during machine learning experiments allows for:

- **Comparing Different Models**: Easily compare various configurations and select the most effective model.
- **Reproducibility**: Detailed logs ensure experiments can be reproduced reliably.

In [8]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Generate some example data
X, y = np.random.rand(100, 5), np.random.randint(0, 2, size=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up MLflow tracking
mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.set_experiment("Machine_Learning_Experiment_Tracking")

# Parameters for logging
params = {"n_estimators": 100, "max_depth": 5}

with mlflow.start_run():
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    accuracy = accuracy_score(y_test, model.predict(X_test))

    mlflow.log_params(params)
    mlflow.log_metric("accuracy", accuracy)
    
    # Create a DataFrame to serve as an input example
    input_example = pd.DataFrame(X_train[:5], columns=[f"feature_{i}" for i in range(X.shape[1])])
    
    # Log the model with an input example
    mlflow.sklearn.log_model(
        model,
        "random_forest_model",
        input_example=input_example
    )
    
    print("Run logged successfully with input example.")

f:\01_Learning\MLflow\venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
f:\01_Learning\MLflow\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(

Run logged successfully with input example.



f:\01_Learning\MLflow\venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


# MLflow Feature Exploration: Model Management and Registry

Managing complex machine learning models is fundamental. MLflow offers robust capabilities for this purpose.

## Introduction to Model Management and Registry

In ML, models undergo frequent updates and iterations. Effective management ensures:

- **Version Control**: Track all iterations without losing historical data.
- **Accessibility**: Models are accessible throughout their lifecycle.
- **Deployment Readiness**: Manage transitions seamlessly from staging to production.

In [10]:
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from sklearn.linear_model import LogisticRegression
import numpy as np

# Set tracking URI for MLflow
mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.set_experiment("Machine_Learning_Model_Management")

# Example training data
x_train = np.random.rand(100, 5)
y_train = np.random.randint(0, 2, size=100)
model = LogisticRegression().fit(x_train, y_train)

# Start MLflow run
with mlflow.start_run():
    model_info = mlflow.sklearn.log_model(model, "logistic_regression_model")
    run_id = mlflow.active_run().info.run_id

# Register the model name if not yet registered
model_name = "Machine_Learning_Logistic_Regression"
client = MlflowClient()

# Check if the model already exists
try:
    client.create_registered_model(model_name)
except mlflow.exceptions.RestException as e:
    # Model already exists
    pass

# Create a new version of the model
model_version = client.create_model_version(
    name=model_name,
    source=model_info.model_uri,
    run_id=run_id
)

print(f"Model version {model_version.version} registered.")

# Transition the model version to 'Staging'
client.transition_model_version_stage(
    name=model_name,
    version=model_version.version,
    stage="Staging"
)

print("Model transitioned to 'Staging' stage.")

2025/01/25 01:49:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model version 1 registered.
Model transitioned to 'Staging' stage.


C:\Users\Z003RUYE\AppData\Local\Temp\ipykernel_2876\387419272.py:42: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


# MLflow Feature Exploration: Performance Comparison

Use MLflow to compare different models and strategies to find the best fit for your needs.

## Importance of Performance Comparison

Exploring multiple models and settings is key:

- **Model Variability**: Different models (linear, ensemble) perform differently.
- **Hyperparameter Tuning**: Systematic optimization affects performance significantly.
- **Metric Monitoring**: Track metrics like accuracy and F1 score.

In [11]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Example data setup
X, y = np.random.rand(100, 5), np.random.randint(0, 2, size=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=10)
}

mlflow.set_experiment("Machine_Learning_Model_Comparison")

for model_name, model in models.items():
    with mlflow.start_run():
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        mlflow.log_param("model_name", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(model, f"models/{model_name.replace(' ', '_')}")
        print(f"{model_name} - Accuracy: {accuracy}")

2025/01/25 01:49:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logistic Regression - Accuracy: 0.55


2025/01/25 01:49:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest - Accuracy: 0.4


# MLflow Feature Exploration: Deployment of Models

Deployment is essential for making models production-ready.

## Why Deployment Matters

That's where your finely tuned models are accessible in production, powering real-world applications.

### Deployment Workflow with MLflow

Steps include:

- **Registering the Model**: Ensure your model is in MLflow's registry.
- **Transitioning Stages**: Move the model from 'Staging' to 'Production'.
- **Scalability Considerations**: Cloud deployments manage larger traffic efficiently.

In [12]:
from mlflow.tracking import MlflowClient

model_name = "Machine_Learning_Logistic_Regression"
client = MlflowClient()

model_version = 1  # Make sure this matches your registered version
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Production"
)

print(f"Model version {model_version} of {model_name} transitioned to 'Production'.")

Model version 1 of Machine_Learning_Logistic_Regression transitioned to 'Production'.


C:\Users\Z003RUYE\AppData\Local\Temp\ipykernel_2876\1548021523.py:7: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


# MLflow Feature Exploration: Model Inferencing

Inference is where we use our production-ready models to make predictions on new data. This is a crucial step in realizing the value of machine learning models in practical applications.

In [13]:
import mlflow.pyfunc
import numpy as np
import pandas as pd

# Set the tracking URI to ensure access to the correct model
mlflow.set_tracking_uri("sqlite:///mlruns.db")

# Define the model name and version
model_name = "Machine_Learning_Logistic_Regression"
model_version = 1  # Ensure this matches the 'Production' version

# Load the production model
print(f"Loading production model version {model_version} of '{model_name}' for inference...")
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# Create a sample input for inference
sample_input = pd.DataFrame(np.random.rand(5, 5), columns=[f"feature_{i}" for i in range(5)])

# Perform inference
predictions = model.predict(sample_input)

print(f"Predictions: {predictions}")

Loading production model version 1 of 'Machine_Learning_Logistic_Regression' for inference...
Predictions: [1 0 1 1 0]


f:\01_Learning\MLflow\venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
